In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os
import datetime
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

#import csv
scores_df=pd.read_csv("../resources/merged_df.csv")
scores_df

,Unnamed: 0,Competition Title,Competition Date,FIRST,LAST,Nation,StartNr,Rank,Qualification,Semifinal,...,Quali_Atempts_Tops,Quali_Attempts_Zones,semi_Tops,semi_Zones,semi_Atempts_Tops,semi_Attempts_Zones,Finals_Tops,Finals_Zones,Final_Atempts_Tops,Final_Attempts_Zones
0,0,European Youth Championships (B) - Brixen (ITA...,20 - 22 September 2019,Naile,MEIGNAN,FRA,15.0,1,6T7z99,3T4z55,...,9,9,3,4,5,5,3,4,8,9
1,1,European Youth Championships (B) - Brixen (ITA...,20 - 22 September 2019,Miriam,FOGU,ITA,27.0,2,4T8z814,3T4z55,...,8,4,3,4,5,5,2,4,2,7
2,2,European Youth Championships (B) - Brixen (ITA...,20 - 22 September 2019,Vanda,MICHALKOVA,SVK,48.0,3,6T7z89,3T3z43,...,8,9,3,3,4,3,2,3,2,3
3,3,European Youth Championships (B) - Brixen (ITA...,20 - 22 September 2019,Lola,SAUTIER,FRA,17.0,4,4T6z69,2T3z89,...,6,9,2,3,8,9,1,3,3,8
4,4,European Youth Championships (B) - Brixen (ITA...,20 - 22 September 2019,Polina,KULAGINA,RUS,39.0,5,4T6z611,3T4z76,...,6,1,3,4,7,6,1,3,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,5427,European Youth Cup (B) - Soure (POR) 2018,5 - 6 May 2018,solene,MOREAU,FRA,44.0,13,5T8z510,NaN,...,3,6,0,0,0,0,0,0,0,0
2429,5428,European Youth Cup (B) - Soure (POR) 2018,5 - 6 May 2018,Lucie,VAILLANT BULTEL,FRA,45.0,14,5T7z99,NaN,...,4,7,0,0,0,0,0,0,0,0
2430,5429,European Youth Cup (B) - Soure (POR) 2018,5 - 6 May 2018,Helena,WALTZ,GER,81.0,15,5T7z1111,NaN,...,5,7,0,0,0,0,0,0,0,0
2431,5430,European Youth Cup (B) - Soure (POR) 2018,5 - 6 May 2018,Marie,COLOT,BEL,10.0,16,5T6z98,NaN,...,0,5,0,0,0,0,0,0,0,0


In [2]:
import pandas as pd
from sqlalchemy import create_engine
# Define the database file path
db_path = 'sqlite:///../Database/climbing.sqlite'
# Create a SQLAlchemy engine
engine = create_engine(db_path)
# Define the SQL query
query = 'SELECT * FROM climbing_table'
# Execute the SQL query and load the results into a pandas DataFrame
scores_df = pd.read_sql(query, engine)
# Print the first 5 rows of the DataFrame
print(scores_df.head())

   Unnamed: 0                                  Competition Title  \
0           0  European Youth Championships (B) - Brixen (ITA...   
1           1  European Youth Championships (B) - Brixen (ITA...   
2           2  European Youth Championships (B) - Brixen (ITA...   
3           3  European Youth Championships (B) - Brixen (ITA...   
4           4  European Youth Championships (B) - Brixen (ITA...   

          Competition Date   FIRST        LAST Nation  StartNr  Rank  \
0  20  - 22 September 2019   Naile     MEIGNAN    FRA     15.0     1   
1  20  - 22 September 2019  Miriam        FOGU    ITA     27.0     2   
2  20  - 22 September 2019   Vanda  MICHALKOVA    SVK     48.0     3   
3  20  - 22 September 2019    Lola     SAUTIER    FRA     17.0     4   
4  20  - 22 September 2019  Polina    KULAGINA    RUS     39.0     5   

  Qualification Semifinal  ... Quali_Atempts_Tops Quali_Attempts_Zones  \
0        6T7z99    3T4z55  ...                  9                    9   
1       4T

In [3]:
scores_df['competitor_id'] = pd.factorize(scores_df['FIRST'] + '_' + scores_df['LAST'])[0] + 1


In [4]:
scores_df.columns

Index(['Unnamed: 0', 'Competition Title', 'Competition Date', 'FIRST', 'LAST',
       'Nation', 'StartNr', 'Rank', 'Qualification', 'Semifinal', 'Final',
       'Category', 'Quali_Tops', 'Quali_Zones', 'Quali_Atempts_Tops',
       'Quali_Attempts_Zones', 'semi_Tops', 'semi_Zones', 'semi_Atempts_Tops',
       'semi_Attempts_Zones', 'Finals_Tops', 'Finals_Zones',
       'Final_Atempts_Tops', 'Final_Attempts_Zones', 'competitor_id'],
      dtype='object')

In [5]:
scores_df=scores_df.drop(columns=["FIRST","LAST","Qualification","Semifinal","Final","Category","Unnamed: 0",'Competition Title'])
scores_df

,Competition Date,Nation,StartNr,Rank,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,semi_Tops,semi_Zones,semi_Atempts_Tops,semi_Attempts_Zones,Finals_Tops,Finals_Zones,Final_Atempts_Tops,Final_Attempts_Zones,competitor_id
0,20 - 22 September 2019,FRA,15.0,1,6,7,9,9,3,4,5,5,3,4,8,9,1
1,20 - 22 September 2019,ITA,27.0,2,4,8,8,4,3,4,5,5,2,4,2,7,2
2,20 - 22 September 2019,SVK,48.0,3,6,7,8,9,3,3,4,3,2,3,2,3,3
3,20 - 22 September 2019,FRA,17.0,4,4,6,6,9,2,3,8,9,1,3,3,8,4
4,20 - 22 September 2019,RUS,39.0,5,4,6,6,1,3,4,7,6,1,3,4,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,5 - 6 May 2018,FRA,44.0,13,1,2,3,6,0,0,0,0,0,0,0,0,923
2429,5 - 6 May 2018,FRA,45.0,14,1,2,4,7,0,0,0,0,0,0,0,0,924
2430,5 - 6 May 2018,GER,81.0,15,1,2,5,7,0,0,0,0,0,0,0,0,501
2431,5 - 6 May 2018,BEL,10.0,16,0,2,0,5,0,0,0,0,0,0,0,0,115


In [6]:
scores_df.nunique()

Competition Date          16
Nation                    63
StartNr                  544
Rank                      67
Quali_Tops                 9
Quali_Zones                9
Quali_Atempts_Tops        10
Quali_Attempts_Zones      22
semi_Tops                  5
semi_Zones                 5
semi_Atempts_Tops         10
semi_Attempts_Zones       18
Finals_Tops                5
Finals_Zones               5
Final_Atempts_Tops        10
Final_Attempts_Zones      19
competitor_id           1127
dtype: int64

In [7]:
scores_df["Competition Date"].value_counts()

11  - 12 May 2019               328
31 August - 2 September 2018    301
20  - 22 September 2019         282
25  - 26 May 2019               258
27  - 28 April 2019             241
2  - 3 June 2018                211
21  - 22 July 2018              193
1  - 4 November 2018            151
5  - 7 September 2019            91
7  - 11 November 2018            87
22  - 31 August 2019             59
15  - 17 June 2018               59
21  - 26 November 2018           55
9  - 16 August 2018              51
9  - 10 March 2019               49
5  - 6 May 2018                  17
Name: Competition Date, dtype: int64

In [8]:
scores_df["Nation"].value_counts()

ITA    161
FRA    150
AUT    122
BEL    114
GER    107
      ... 
KGZ      3
RSA      2
PER      2
NEP      2
MEX      1
Name: Nation, Length: 63, dtype: int64

In [9]:
# nation_series = scores_df["Nation"].value_counts()
# nation_names = nation_series.index.tolist()

In [10]:
# #Create cutoff value for the countries
# nations_to _replace =[]


In [11]:
encoded_df = pd.get_dummies(scores_df, columns=["Nation"])
encoded_df['competition_id'] = pd.factorize(encoded_df["Competition Date"])[0] + 1
encoded_df

,Competition Date,StartNr,Rank,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,semi_Tops,semi_Zones,semi_Atempts_Tops,...,Nation_SUI,Nation_SVK,Nation_SWE,Nation_THA,Nation_TPE,Nation_TUR,Nation_UKR,Nation_USA,Nation_VEN,competition_id
0,20 - 22 September 2019,15.0,1,6,7,9,9,3,4,5,...,0,0,0,0,0,0,0,0,0,1
1,20 - 22 September 2019,27.0,2,4,8,8,4,3,4,5,...,0,0,0,0,0,0,0,0,0,1
2,20 - 22 September 2019,48.0,3,6,7,8,9,3,3,4,...,0,1,0,0,0,0,0,0,0,1
3,20 - 22 September 2019,17.0,4,4,6,6,9,2,3,8,...,0,0,0,0,0,0,0,0,0,1
4,20 - 22 September 2019,39.0,5,4,6,6,1,3,4,7,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,5 - 6 May 2018,44.0,13,1,2,3,6,0,0,0,...,0,0,0,0,0,0,0,0,0,16
2429,5 - 6 May 2018,45.0,14,1,2,4,7,0,0,0,...,0,0,0,0,0,0,0,0,0,16
2430,5 - 6 May 2018,81.0,15,1,2,5,7,0,0,0,...,0,0,0,0,0,0,0,0,0,16
2431,5 - 6 May 2018,10.0,16,0,2,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,16


In [12]:
encoded_df.dtypes

Competition Date     object
StartNr             float64
Rank                  int64
Quali_Tops            int64
Quali_Zones           int64
                     ...   
Nation_TUR            uint8
Nation_UKR            uint8
Nation_USA            uint8
Nation_VEN            uint8
competition_id        int64
Length: 80, dtype: object

In [13]:
import re

def fix_date(date_string):
    regex_pattern = r"^(\d{1,2})\s-\s\d{1,2}\s(\w+)\s(\d{4})"
    match_obj = re.match(regex_pattern, date_string)

    if match_obj:
        output_str = f"{match_obj.group(1)} {match_obj.group(2)} {match_obj.group(3)}"
        print(output_str)
    else:
        print("No match found")
    return output_str

In [14]:
dates = encoded_df['Competition Date'].str.extractall(r"^(\d{1,2})\s*(-\s*(\d{1,2})\s+)?(\w+)\s+(\d{4})?")
dates = pd.DataFrame(dates)
dates = dates.reset_index().drop(columns=["match", "level_0",1,2]).reset_index()
dates = dates.drop(columns=["index"])
dates.columns=["day","month","year"]
dates["month"]=pd.to_datetime(dates["month"],format="%B")
dates["month"]=dates["month"].dt.month
dates = dates.fillna(9)
dates.astype({'day':'int64','year':'int64'}).dtypes
dates


,day,month,year
0,20,9,2019
1,20,9,2019
2,20,9,2019
3,20,9,2019
4,20,9,2019
...,...,...,...
2428,5,5,2018
2429,5,5,2018
2430,5,5,2018
2431,5,5,2018


In [15]:
merger_dates_df = pd.merge(encoded_df, dates, left_index=True, right_index=True)
df = merger_dates_df.drop(columns=['Competition Date'])

In [16]:
df

,StartNr,Rank,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,semi_Tops,semi_Zones,semi_Atempts_Tops,semi_Attempts_Zones,...,Nation_THA,Nation_TPE,Nation_TUR,Nation_UKR,Nation_USA,Nation_VEN,competition_id,day,month,year
0,15.0,1,6,7,9,9,3,4,5,5,...,0,0,0,0,0,0,1,20,9,2019
1,27.0,2,4,8,8,4,3,4,5,5,...,0,0,0,0,0,0,1,20,9,2019
2,48.0,3,6,7,8,9,3,3,4,3,...,0,0,0,0,0,0,1,20,9,2019
3,17.0,4,4,6,6,9,2,3,8,9,...,0,0,0,0,0,0,1,20,9,2019
4,39.0,5,4,6,6,1,3,4,7,6,...,0,0,0,0,0,0,1,20,9,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,44.0,13,1,2,3,6,0,0,0,0,...,0,0,0,0,0,0,16,5,5,2018
2429,45.0,14,1,2,4,7,0,0,0,0,...,0,0,0,0,0,0,16,5,5,2018
2430,81.0,15,1,2,5,7,0,0,0,0,...,0,0,0,0,0,0,16,5,5,2018
2431,10.0,16,0,2,0,5,0,0,0,0,...,0,0,0,0,0,0,16,5,5,2018


In [17]:
import numpy as np
 
mask = np.column_stack([encoded_df['Competition Date'].str.contains(r"March", na=False) for col in encoded_df])
encoded_df.loc[mask.any(axis=1)]

,Competition Date,StartNr,Rank,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,semi_Tops,semi_Zones,semi_Atempts_Tops,...,Nation_SUI,Nation_SVK,Nation_SWE,Nation_THA,Nation_TPE,Nation_TUR,Nation_UKR,Nation_USA,Nation_VEN,competition_id
1259,9 - 10 March 2019,201.0,1,0,4,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,7
1260,9 - 10 March 2019,214.0,2,0,3,0,4,0,0,0,...,0,0,0,0,1,0,0,0,0,7
1261,9 - 10 March 2019,222.0,3,0,3,0,6,0,0,0,...,0,0,0,1,0,0,0,0,0,7
1262,9 - 10 March 2019,215.0,4,0,2,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1263,9 - 10 March 2019,219.0,5,0,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1264,9 - 10 March 2019,208.0,6,8,8,9,8,4,4,4,...,0,0,0,0,0,0,0,0,0,7
1265,9 - 10 March 2019,211.0,7,8,8,1,10,3,4,9,...,0,0,0,0,0,0,0,0,0,7
1266,9 - 10 March 2019,216.0,8,5,6,9,7,3,4,1,...,0,0,0,0,0,0,0,0,0,7
1267,9 - 10 March 2019,207.0,9,6,6,1,7,3,4,9,...,0,0,0,0,0,0,0,0,0,7
1268,9 - 10 March 2019,202.0,10,7,8,9,9,4,4,6,...,0,0,0,0,0,0,0,0,0,7


In [18]:
import numpy as np

# Assume your rank column is named 'rank' in a pandas DataFrame named 'df'
top_n = 6  # Define the number of top ranks you want to set to 1

# Create a new column named 'rank_top' with all values initialized to 0
df['rank_top'] = np.zeros(len(df))

# Set the top 'top_n' ranks to 1 in the 'rank_top' column
df.loc[df['Rank'] <= top_n, 'rank_top'] = 1

In [19]:
df=df.drop(columns=['Rank'])
df

,StartNr,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,semi_Tops,semi_Zones,semi_Atempts_Tops,semi_Attempts_Zones,Finals_Tops,...,Nation_TPE,Nation_TUR,Nation_UKR,Nation_USA,Nation_VEN,competition_id,day,month,year,rank_top
0,15.0,6,7,9,9,3,4,5,5,3,...,0,0,0,0,0,1,20,9,2019,1.0
1,27.0,4,8,8,4,3,4,5,5,2,...,0,0,0,0,0,1,20,9,2019,1.0
2,48.0,6,7,8,9,3,3,4,3,2,...,0,0,0,0,0,1,20,9,2019,1.0
3,17.0,4,6,6,9,2,3,8,9,1,...,0,0,0,0,0,1,20,9,2019,1.0
4,39.0,4,6,6,1,3,4,7,6,1,...,0,0,0,0,0,1,20,9,2019,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,44.0,1,2,3,6,0,0,0,0,0,...,0,0,0,0,0,16,5,5,2018,0.0
2429,45.0,1,2,4,7,0,0,0,0,0,...,0,0,0,0,0,16,5,5,2018,0.0
2430,81.0,1,2,5,7,0,0,0,0,0,...,0,0,0,0,0,16,5,5,2018,0.0
2431,10.0,0,2,0,5,0,0,0,0,0,...,0,0,0,0,0,16,5,5,2018,0.0


In [29]:
df = df.dropna()
df.columns

Index(['StartNr', 'Quali_Tops', 'Quali_Zones', 'Quali_Atempts_Tops',
       'Quali_Attempts_Zones', 'semi_Tops', 'semi_Zones', 'semi_Atempts_Tops',
       'semi_Attempts_Zones', 'Finals_Tops', 'Finals_Zones',
       'Final_Atempts_Tops', 'Final_Attempts_Zones', 'competitor_id',
       'Nation_ARG', 'Nation_AUS', 'Nation_AUT', 'Nation_BEL', 'Nation_BLR',
       'Nation_BRA', 'Nation_BUL', 'Nation_CAM', 'Nation_CAN', 'Nation_CHI',
       'Nation_CHN', 'Nation_CRO', 'Nation_CZE', 'Nation_DEN', 'Nation_ECU',
       'Nation_ESP', 'Nation_FIN', 'Nation_FRA', 'Nation_GBR', 'Nation_GER',
       'Nation_GRE', 'Nation_GUA', 'Nation_HKG', 'Nation_HUN', 'Nation_INA',
       'Nation_IND', 'Nation_IRI', 'Nation_IRL', 'Nation_ISR', 'Nation_ITA',
       'Nation_JPN', 'Nation_KAZ', 'Nation_KGZ', 'Nation_KOR', 'Nation_LAT',
       'Nation_LTU', 'Nation_LUX', 'Nation_MAC', 'Nation_MAS', 'Nation_MEX',
       'Nation_NED', 'Nation_NEP', 'Nation_NOR', 'Nation_NZL', 'Nation_PER',
       'Nation_PHI', 'Nation

In [30]:
encoded = pd.get_dummies(df, columns=["competition_id","year","competitor_id","month"])
encoded

,StartNr,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,semi_Tops,semi_Zones,semi_Atempts_Tops,semi_Attempts_Zones,Finals_Tops,...,competitor_id_1126,competitor_id_1127,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_11
0,15.0,6,7,9,9,3,4,5,5,3,...,0,0,0,0,0,0,0,0,1,0
1,27.0,4,8,8,4,3,4,5,5,2,...,0,0,0,0,0,0,0,0,1,0
2,48.0,6,7,8,9,3,3,4,3,2,...,0,0,0,0,0,0,0,0,1,0
3,17.0,4,6,6,9,2,3,8,9,1,...,0,0,0,0,0,0,0,0,1,0
4,39.0,4,6,6,1,3,4,7,6,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,44.0,1,2,3,6,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2429,45.0,1,2,4,7,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2430,81.0,1,2,5,7,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2431,10.0,0,2,0,5,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [31]:
# Split our preprocessed data into our features and target arrays
y = encoded.rank_top.values
X = encoded.drop(columns="rank_top").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

In [32]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [33]:
len(X_train_scaled[0])

1217

In [34]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  51
hidden_nodes_layer2 = 76
hidden_nodes_layer3 = 91
hidden_nodes_layer4 = 6




nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 51)                62118     
                                                                 
 dense_7 (Dense)             (None, 76)                3952      
                                                                 
 dense_8 (Dense)             (None, 76)                5852      
                                                                 
 dense_9 (Dense)             (None, 91)                7007      
                                                                 
 dense_10 (Dense)            (None, 6)                 552       
                                                                 
 dense_11 (Dense)            (None, 1)                 7         
                                                                 
Total params: 79,488
Trainable params: 79,488
Non-trai

In [35]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [36]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
53/53 [==============================] - 1s 3ms/step - loss: 0.5105 - accuracy: 0.7896
Epoch 2/100
53/53 [==============================] - 0s 4ms/step - loss: 0.3314 - accuracy: 0.8489
Epoch 3/100
53/53 [==============================] - 0s 7ms/step - loss: 0.2240 - accuracy: 0.8645
Epoch 4/100
53/53 [==============================] - 0s 5ms/step - loss: 0.1441 - accuracy: 0.9424
Epoch 5/100
53/53 [==============================] - 0s 4ms/step - loss: 0.1001 - accuracy: 0.9622
Epoch 6/100
53/53 [==============================] - 0s 4ms/step - loss: 0.0735 - accuracy: 0.9694
Epoch 7/100
53/53 [==============================] - 0s 4ms/step - loss: 0.0400 - accuracy: 0.9868
Epoch 8/100
53/53 [==============================] - 0s 4ms/step - loss: 0.0330 - accuracy: 0.9868
Epoch 9/100
53/53 [==============================] - 0s 4ms/step - loss: 0.0176 - accuracy: 0.9952
Epoch 10/100
53/53 [==============================] - 0s 4ms/step - loss: 0.0101 - accuracy: 0.9964
Epoch 11/

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

18/18 - 0s - loss: 2.5334 - accuracy: 0.7630 - 181ms/epoch - 10ms/step
Loss: 2.533405303955078, Accuracy: 0.7630161643028259


In [41]:
# Making predictions using the testing data
predictions = nn.predict(X_test_scaled)

18/18 [==============================] - 0s 3ms/step


In [56]:
predictions
predictions = np.round(predictions, 0)
# prediction = [np.round(elem, 0) for elem in predictions]
# prediction
predictions

array([[0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [57]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [58]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,381,92
Actual 1,40,44


Accuracy Score : 0.7630161579892281
Classification Report
              precision    recall  f1-score   support

         0.0       0.90      0.81      0.85       473
         1.0       0.32      0.52      0.40        84

    accuracy                           0.76       557
   macro avg       0.61      0.66      0.63       557
weighted avg       0.82      0.76      0.78       557



In [60]:
nn.save('ClimberNeuralNetwork.h5')